# ストレージ・メモリ・実行速度の向上

- 利用する特徴を減らすことで、ストレージ・メモリの使用量を減らしたり、モデルを早く動かせる
- 正則化が適用できないモデルでは過学習の防止に役立つが、正則化が適用できるモデルでは過学習防止には正則化を用いる

## 特徴選択(feature selection)

全ての特徴から任意の個数の特徴を選び出す。選び出された特徴自体は元のまま。

### Lasso回帰

[Lasso回帰](/notebooks/jupyter/basic/improvement/regularization/lasso_regression.ipynb)は特徴選択に利用可能。

### Recursive Feature Elimination

重みの小さいものから順に削除していき、特徴の重要度をランク付けする。

In [ ]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.feature_selection import RFE
from sklearn.linear_model import Perceptron

loader = load_breast_cancer()
X, y = loader.data, loader.target
model = Perceptron()
selector = RFE(estimator=model, n_features_to_select=5, step=1)
X_new = selector.fit_transform(X, y)
print('Original Feature: {origin}, Selected Feature: {selected}'.format(origin=X.shape[1], selected=X_new.shape[1]))
ranking = selector.ranking_
rank_order = np.argsort(ranking)
for i, rank_id in enumerate(rank_order):
    print('Feature Importance Rank {rank:>2}: Feature {feature}'.format(rank=ranking[rank_id], feature=rank_id))

### ランダムフォレスト

[ランダムフォレストを利用して特徴の重要度を導く方法](random_forest_feature_importance.ipynb)もある。

## 次元削減(Dimensionality Reduction)

多次元の特徴を、できるだけ情報を保持したまま低次元の特徴に落とす。得られた特徴は元の特徴とは異なる。

### 主成分分析(Principal Component Analysis, PCA)

- 多次元の相関のある特徴を、低次元の相関の少ない特徴に落とす手法
- データのスケールに対して敏感なので、実施前に各特徴を標準化しておかなければならない

#### 使用方法

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# データ準備
np.random.seed(0)

X, _ = make_classification(n_samples=300, n_features=3, n_informative=2, n_redundant=1, n_clusters_per_class=1, random_state=0)
X += np.random.normal(scale=.2, size=X.shape)
x, y, z = X[:, 0], X[:, 1], X[:, 2]

# 主成分分析
pca = PCA(n_components=2)
pca.fit(X)

In [ ]:
X_new = pca.transform(X)

# 可視化
x_pca, y_pca = X_new[:, 0], X_new[:, 1]
margin = .5
x_pca_min = y_pca_min = min(x_pca.min(), y_pca.min()) - margin
x_pca_max = y_pca_max = max(x_pca.max(), y_pca.max()) + margin
xx_pca, yy_pca = np.meshgrid([x_pca_min, x_pca_max], [y_pca_min, y_pca_max])
rect_pca = np.c_[xx_pca.ravel(), yy_pca.ravel()]
rect = pca.inverse_transform(rect_pca)
x_rect, y_rect, z_rect = rect[:, 0], rect[:, 1], rect[:, 2]
x_rect.shape = (2, 2)
y_rect.shape = (2, 2)
z_rect.shape = (2, 2)

def plot(fig_id, elev, azim):
    fig = plt.figure(fig_id)
    ax = Axes3D(fig, elev=elev, azim=azim)
    ax.scatter(x, y, z, c='blue', marker='+')
    ax.plot_surface(x_rect, y_rect, z_rect, color='red', alpha=.6)
    ax.set_xticklabels(())
    ax.set_yticklabels(())
    ax.set_zticklabels(())

plot(0, -55, 120)
plot(1, -145, 120)

plt.figure(2, figsize=(5, 5))
plt.scatter(x_pca, y_pca, c='blue', marker='+')
plt.xticks(())
plt.yticks(())

plt.show()

In [ ]:
def draw_components(model):
    components = model.components_
    n_components = components.shape[0]
    n_features = components.shape[1]
    features = np.arange(n_features)
    x_min, x_max = -1, n_features
    y_min, y_max = components.min() - .05, components.max() + .05

    fig = plt.figure(figsize=(8, 2 * n_components))

    for i in range(n_components):
        fig.add_subplot(n_components, 1, i + 1)
        plt.title('Component {n}'.format(n=i))
        plt.bar(features, components[i, :], align='center')

        if i < n_components - 1:
            plt.xticks(())
        else:
            labels = ['feature {n}'.format(n=n) for n in features]
            plt.xticks(features, labels, rotation='vertical')

        plt.xlim(x_min, x_max)
        plt.ylim(y_min, y_max)

    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0, hspace=.5)
    plt.show()

In [ ]:
from sklearn.datasets import load_diabetes

# データ準備
loader = load_diabetes()
X = loader.data

# 主成分分析
pca = PCA().fit(X)

# 各成分の可視化
print('Explained variance ratio: ', ['{v:.3f}'.format(v=v) for v in pca.explained_variance_ratio_])

for n in range(X.shape[1], 0, -1):
    print('New dimensions: {n}, Left variance: {v:.3f}'.format(n=n, v=pca.explained_variance_ratio_[:n].sum()))

draw_components(pca)

### 線形判別分析(Linear Discriminant Analysis)

- 教師あり次元削減に利用可能な手法
- 訓練データの異なるラベル同士が最も離れるように低次元の特徴に落とす

#### 使用方法

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# データ準備
X, y = make_blobs(centers=([[1, 0]]), cluster_std=.35, random_state=0)
X = np.concatenate((X, -1 * X), axis=0)
X = StandardScaler().fit_transform(X)
y = np.concatenate((y, np.ones(len(y))), axis=0)

# 線形判別分析と主成分分析の比較
fig, axes = plt.subplots(1, 3, figsize=(9, 3))

titles = ['Original Data', 'LDA', 'PCA']

for i, ax in enumerate(axes):
    if i == 0:
        xx, yy = X[:, 0], X[:, 1]
    elif i == 1:
        xx = LinearDiscriminantAnalysis(n_components=1).fit_transform(X, y)
        yy = np.zeros_like(xx)
    else:
        xx = PCA(n_components=1).fit_transform(X)
        yy = np.zeros_like(xx)
        yy[y == 0] = .0005
        yy[y == 1] = -.0005
    ax.set_title(titles[i])
    ax.scatter(xx, yy, c=y, cmap='bwr', marker='x')
    ax.set_xticks(())
    ax.set_yticks(())

plt.show()

In [ ]:
# データ準備
X, y = loader.data, loader.target

# 線形判別分析
lda = LinearDiscriminantAnalysis()
lda.fit(X, y)

In [ ]:
# 各成分の可視化
print('Explained variance ratio: ', ['{v:.3f}'.format(v=v) for v in lda.explained_variance_ratio_])

for n in range(X.shape[1], 0, -1):
    print('New dimensions: {n}, Left variance: {v:.3f}'.format(n=n, v=lda.explained_variance_ratio_[:n].sum()))

### 潜在的ディリクレ配分法(Latent Dirichlet Allocation, LDA)

- 文書の特徴抽出・次元削減に利用可能な手法
- 文書に含まれる単語から、文書のトピック(話題・カテゴリ)を確率的に推測する

#### 使用方法

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# データ準備
remove = ('headers', 'footers', 'quotes')

datasets = fetch_20newsgroups(shuffle=True, random_state=0, remove=('headers', 'footers', 'quotes'))
data = datasets.data

print(data[0])

In [ ]:
vectorizer = CountVectorizer(stop_words='english', max_df=0.95, min_df=2, max_features=2 ** 10)
X = vectorizer.fit_transform(data)

X.shape

In [ ]:
print(X[0])

In [ ]:
# LDA
lda = LatentDirichletAllocation(n_topics=10, max_iter=5, learning_method='online',
                                learning_offset=50., n_jobs=-1, random_state=0)
lda.fit(X)

In [ ]:
# 各トピックの可視化
feature_names = vectorizer.get_feature_names()

for idx, topic in enumerate(lda.components_):
    print('Topic {n}: {topic}'.format(n=idx, topic=[feature_names[i] for i in topic.argsort()[:-11:-1]]))

In [ ]:
# 文書への適用
import numpy as np

doc_id = 0
topics = lda.transform(X[0])[0]

print('Document {i}'.format(i=doc_id))
for i, p in enumerate(topics):
    print('Topic {i}: {p:.3f}'.format(i=i, p=p))